In [1]:
#Process log file into a data frame format

import pandas as pd
import re
import datetime
import os
import numpy as np

date_ = []
time_ = []
pid_ = []
level_ = []
component_ = []
content_ = []
contentType_ = []
parsedData = []

# Use 'HDFS_2k.log' as a test case
# 'HDFS_1.tar' may exceed available memory if run locally
# 'HDFS_1.tar' can be retrieved using the following link 'https://zenodo.org/record/3227177/files/HDFS_2.tar.gz?download=1'
with open("HDFS_2k.log", "r") as file:
# with open("HDFS_1.tar", "r") as file:
    for line in file:

        #Get all odered data
        columns1_4 = line.split(":", 1)[0].strip()

        #Remove all rows with an inproper format
        try:
            columns1_4.split(' ')[0]
            columns1_4.split(' ')[1]
            columns1_4.split(' ')[2]
            columns1_4.split(' ')[3]
            columns1_4.split(' ')[4]
            date = columns1_4.split(' ')[0]
            time = columns1_4.split(' ')[1]
            pid = columns1_4.split(' ')[2]
            level = columns1_4.split(' ')[3]
            component = columns1_4.split(' ')[4]
        except:
            pass


        # Get colon with mixed data 
        content = line.split(":", 1)[1].strip()  # after colon
        list1 = (line.split(":", 1)[1].strip()).split(" ", 3)[0:2]
        contentType=' '.join(list1)
        
        date_.append(date)
        time_.append(time)
        pid_.append(pid)
        level_.append(level)
        component_.append(component)
        content_.append(content)
        contentType_.append(contentType)

# Collect and filter data
parsedData = pd.DataFrame(
    {
        "Date": date_,
        "Time": time_,
        "PID": pid_,
        "Level": level_,
        "Component": component_,
        "Content": content_,
        "TypeOfContent": contentType_,
    }
)
parsedData["datetime_id"] = parsedData["Date"] + parsedData["Time"]
parsedData["Tag"] = parsedData["TypeOfContent"]

parsedData1 = parsedData[
    (parsedData["Tag"] == "PacketResponder 0")
    | (parsedData["Tag"] == "PacketResponder 1")
    | (parsedData["Tag"] == "PacketResponder 2")
    | (parsedData["Tag"] == "BLOCK* NameSystem.addStoredBlock:")
    | (parsedData["Tag"] == "BLOCK* NameSystem.delete:")
    | (parsedData["Tag"] == "BLOCK* NameSystem.allocateBlock:")
    | (parsedData["Tag"] == "BLOCK* ask")
    | (parsedData["Tag"] == "Received block")
    | (parsedData["Tag"] == "Receiving block")
    | (parsedData["Tag"] == "Verification succeeded")
    | (parsedData["Tag"] == "Deleting block")
]
parsedData2 = parsedData[
    (parsedData["Tag"] != "PacketResponder 0")
    & (parsedData["Tag"] != "PacketResponder 1")
    & (parsedData["Tag"] != "PacketResponder 2")
    & (parsedData["Tag"] != "BLOCK* NameSystem.addStoredBlock:")
    & (parsedData["Tag"] != "BLOCK* NameSystem.delete:")
    & (parsedData["Tag"] != "BLOCK* NameSystem.allocateBlock:")
    & (parsedData["Tag"] != "BLOCK* ask")
    & (parsedData["Tag"] != "Received block")
    & (parsedData["Tag"] != "Receiving block")
    & (parsedData["Tag"] != "Verification succeeded")
    & (parsedData["Tag"] != "Deleting block")
]
parsedData2["Tag"] = parsedData2["Tag"].astype(str)
parsedData2["TypeOfContent"] = parsedData2["TypeOfContent"].astype(str)
parsedData2_fix = parsedData2[parsedData2["Tag"].str.contains("exception")]
parsedData2_fix["Tag"] = "Got exception"
parsedData2_fix["TypeOfContent"] = "Got exception"
parsedData2_fix_2 = parsedData2[parsedData2["TypeOfContent"].str.contains("Served")]
parsedData2_fix_2["Tag"] = "Served"
parsedData2_fix_2["TypeOfContent"] = "Served"
parsedData2_fix_3 = parsedData2[parsedData2["TypeOfContent"].str.contains("Starting")]
parsedData2_fix_3["Tag"] = "Starting thread"
parsedData2_fix_3["TypeOfContent"] = "Starting thread"

frames = [parsedData1, parsedData2_fix, parsedData2_fix_2, parsedData2_fix_3]

parsedData = pd.concat(frames)
parsedData["Date"] = "20" + parsedData["Date"]
parsedData["Date"] = pd.to_datetime(parsedData["Date"], format="%Y%m%d", errors="ignore")     
        
parsedData_2 = parsedData.sort_values(by=['Date', 'Time'])
print(parsedData_2)

#Export to .txt due to file size limit
parsedData_2.to_csv('HDFS_2k-parsed_3.txt', sep='\t', index=False)

           Date    Time    PID Level                     Component  \
0    2008-11-09  203615    148  INFO  dfs.DataNode$PacketResponder   
2    2008-11-09  203807    222  INFO  dfs.DataNode$PacketResponder   
3    2008-11-09  204005     35  INFO              dfs.FSNamesystem   
4    2008-11-09  204015    308  INFO  dfs.DataNode$PacketResponder   
5    2008-11-09  204106    329  INFO  dfs.DataNode$PacketResponder   
...         ...     ...    ...   ...                           ...   
1996 2008-11-11  101621  24902  INFO      dfs.DataNode$DataXceiver   
1997 2008-11-11  101735  26595  INFO  dfs.DataNode$PacketResponder   
1998 2008-11-11  101804  26494  INFO      dfs.DataNode$DataXceiver   
1999 2008-11-11  101954  26414  INFO  dfs.DataNode$PacketResponder   
2000 2008-11-11  102017  26347  INFO      dfs.DataNode$DataXceiver   

                                                Content  \
0     PacketResponder 1 for block blk_38865049064139...   
2     PacketResponder 0 for block blk_-69

<ipython-input-1-888ee5e29a88>:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsedData2["Tag"] = parsedData2["Tag"].astype(str)
<ipython-input-1-888ee5e29a88>:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  parsedData2["TypeOfContent"] = parsedData2["TypeOfContent"].astype(str)
<ipython-input-1-888ee5e29a88>:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [2]:
df = pd.read_csv('HDFS_2k-parsed_3.txt', delimiter = "\t")
df.dropna(inplace=True)
df.tail()

,Date,Time,PID,Level,Component,Content,TypeOfContent,datetime_id,Tag
1996,2008-11-11,101621,24902,INFO,dfs.DataNode$DataXceiver,Receiving block blk_4198733391373026104 src: /...,Receiving block,81111101621,Receiving block
1997,2008-11-11,101735,26595,INFO,dfs.DataNode$PacketResponder,Received block blk_-5815145248455404269 of siz...,Received block,81111101735,Received block
1998,2008-11-11,101804,26494,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-295306975763175640 src: /...,Receiving block,81111101804,Receiving block
1999,2008-11-11,101954,26414,INFO,dfs.DataNode$PacketResponder,PacketResponder 0 for block blk_52257196770490...,PacketResponder 0,81111101954,PacketResponder 0
2000,2008-11-11,102017,26347,INFO,dfs.DataNode$DataXceiver,Receiving block blk_4343207286455274569 src: /...,Receiving block,81111102017,Receiving block


In [3]:
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df["Level_code"] = ord_enc.fit_transform(df[["Level"]])
print(ord_enc.categories_)
df["Component_code"] = ord_enc.fit_transform(df[["Component"]])
print(ord_enc.categories_)
df["TypeOfContent_code"] = ord_enc.fit_transform(df[["TypeOfContent"]])
print(ord_enc.categories_)
df["Tag_code"] = ord_enc.fit_transform(df[["Tag"]])
print(ord_enc.categories_)

[array(['INFO', 'WARN'], dtype=object)]
[array(['dfs.DataBlockScanner', 'dfs.DataNode', 'dfs.DataNode$DataXceiver',
       'dfs.DataNode$PacketResponder', 'dfs.FSDataset',
       'dfs.FSNamesystem'], dtype=object)]
[array(['BLOCK* NameSystem.addStoredBlock:',
       'BLOCK* NameSystem.allocateBlock:', 'BLOCK* NameSystem.delete:',
       'BLOCK* ask', 'Deleting block', 'Got exception',
       'PacketResponder 0', 'PacketResponder 1', 'PacketResponder 2',
       'Received block', 'Receiving block', 'Served', 'Starting thread',
       'Verification succeeded'], dtype=object)]
[array(['BLOCK* NameSystem.addStoredBlock:',
       'BLOCK* NameSystem.allocateBlock:', 'BLOCK* NameSystem.delete:',
       'BLOCK* ask', 'Deleting block', 'Got exception',
       'PacketResponder 0', 'PacketResponder 1', 'PacketResponder 2',
       'Received block', 'Receiving block', 'Served', 'Starting thread',
       'Verification succeeded'], dtype=object)]


In [4]:
#Remove block without block ID
df.drop(df.tail(1).index,inplace=True)

# add block ID to dataframe
listBlocks = [itm[0] for itm in df['Content'].str.findall(r'\sblk\D+\d+') if len(itm)>0]
listBlocksStriped = [i.strip(' ') for i in listBlocks]
df['blkID'] = listBlocksStriped

In [5]:
# Group rows with identical blkIDs
df_comp = df[['Tag_code','blkID']]
df_comp['Tag_code'] = df_comp['Tag_code'].astype(int).astype(str)
df_merged = df_comp[['blkID','Tag_code']].groupby('blkID')['Tag_code'].apply(' '.join).reset_index()

<ipython-input-5-1cb5137f3695>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comp['Tag_code'] = df_comp['Tag_code'].astype(int).astype(str)


In [6]:
# Append column to label anomallies and normal data logs
AnomallyLabel = pd.read_csv('AnomallyLabels.csv')
AnomallyLabel = AnomallyLabel['BlockId,Label'].str.split(",",expand=True,)
AnomallyLabel.rename(columns={0:'blockID', 1:"Label"}, inplace = True)
dfLabelled = pd.merge(df_merged,AnomallyLabel, left_on = 'blkID', right_on='blockID')
df_initialAnalysis = pd.merge(df,AnomallyLabel, left_on = 'blkID', right_on='blockID')
df_initialAnalysis.to_csv('HDFS_2k-parsed_3-initialAnalysis.csv', index=False)
dfLabelled = dfLabelled.drop(columns=['blockID'])
dfLabelled.head()

,blkID,Tag_code,Label
0,blk_-1030832046197982436,2,Normal
1,blk_-1046472716157313227,4,Normal
2,blk_-1049340855430710153,8,Normal
3,blk_-1055254430948037872,0,Normal
4,blk_-1067234447809438340,6,Normal


In [7]:
#Convert dataframe into count matrix
from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
df_filt = dfLabelled[['blkID', 'Tag_code','Label']]
df_filt["Label"] = ord_enc.fit_transform(df_filt[["Label"]]).astype(int)
df_filt['Tag_code'] = [' '.join(map(str, l)) for l in dfLabelled['Tag_code']]
df_filt.tail()

,blkID,Tag_code,Label
1988,blk_92946806844541836,4,1
1989,blk_931297048899943584,1 0,1
1990,blk_939695665085237271,0,1
1991,blk_955083461510572332,9,1
1992,blk_995094303184954394,1 0,1


In [8]:
df_NN=df_filt.drop(['blkID'], axis=1)
df_NN.rename(columns = {'Tag_code':'review','Label':'sentiment'}, inplace = True)
df_NN.to_csv('HDFS_2k-parsed_3-compl.csv', index=False)

In [9]:
# Create design matrix for 'R' neuro net and PCA_RF file
def createCol(inputDF, outputDF, colName):
    colList = list(set((' '.join(inputDF[colName].astype(str).unique().tolist())).split(' ')))
    colNum = [int(num) for num in colList]
    colNum.sort()
    for col in colNum:
        outputDF['tag_code%s' % col] = 0

    for i in range(len(inputDF)):
        valueSplit = inputDF[colName].iloc[i].split(' ')
        counts = [[str(x),valueSplit.count(str(x))] for x in colNum]
        for val in counts:
            outputDF.loc[i,'tag_code%s' % val[0]] = val[1]

    return outputDF

In [10]:
df_NNR = df_initialAnalysis[['blkID','Label']]
createCol(df_merged,df_NNR,'Tag_code')
df_NNR.to_csv('R_NN.txt', sep='\t', index=False)

<ipython-input-9-8ea679390d3c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outputDF['tag_code%s' % col] = 0
